## Import

In [ ]:
import json
import numpy as np
import tensorflow.keras as keras
import music21 as m21

## MelodyGenerator Class

In [ ]:


class MelodyGenerator:
    def __init__(self, model_path="model.h5"):
        self.model_path = model_path
        self.model = keras.models.load_model(model_path)

        with open("/mapping.json", "r") as fp:
            self._mappings = json.load(fp)

        self._start_symbols = ["/"] * 64


    def generate_melody(self, start_melody, num_steps, max_sequence_length, temperature):

        start_melody = start_melody.split()
        melody = start_melody
        start_melody = self._start_symbols + start_melody

        # map start_melody to int
        start_melody = [self._mappings[symbol] for symbol in start_melody]

        for _ in range(num_steps):

            # limit the start_melody to max_sequence_length
            start_melody = start_melody[-max_sequence_length:]

            # one-hot encode the start_melody
            onehot_start_melody = keras.utils.to_categorical(start_melody, num_classes=len(self._mappings))
            onehot_start_melody = onehot_start_melody[np.newaxis, ...]

            # make a prediction
            probabilities = self.model.predict(onehot_start_melody)[0]
            # [0.1, 0.2, 0.1, 0.6] -> 1
            output_int = self.resample(probabilities, temperature)

            # update start_melody
            start_melody.append(output_int)

            # map int to our encoding
            output_symbol = [k for k, v in self._mappings.items() if v == output_int][0]

            # check whether we're at the end of a melody
            if output_symbol == "/":
                break

            # update melody
            melody.append(output_symbol)

        return melody


    def resample(self, probabilites, temperature):

        predictions = np.log(probabilites) / temperature
        probabilites = np.exp(predictions) / np.sum(np.exp(predictions))

        choices = range(len(probabilites)) # [0, 1, 2, 3]
        index = np.random.choice(choices, p=probabilites)

        return index


    def save_melody(self, melody, step_duration=0.25, format="midi", file_name="mel.mid"):

        # create a music21 stream
        stream = m21.stream.Stream()

        start_symbol = None
        step_counter = 1

        
        for i, symbol in enumerate(melody): 
            if symbol != "_" or i + 1 == len(melody):
                if start_symbol is not None:
                    quarter_length_duration = step_duration * step_counter # 0.25 * 4 = 1

                    # rest
                    if start_symbol == "r":
                        m21_event = m21.note.Rest(quarterLength=quarter_length_duration)

                    # note
                    else:
                        m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)

                    stream.append(m21_event)
                    step_counter = 1

                start_symbol = symbol

            # "_"
            else:
                step_counter += 1

        # stream to a midi file
        stream.write(format, file_name)

## Example

In [ ]:

mg = MelodyGenerator()
start_melody = "67 _ 67 _ 67 _ _ 65 64 _ 64 _ 64 _ _"
start_melody2 = "67 _ _ _ _ _ 65 _ 64 _ 62 _ 60 _ _ _"
melody = mg.generate_melody(start_melody, 500, 64, 0.3)
print(melody)